In [14]:
%pip install -q -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [9]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="/notebooks/stage.env")

import os

In [16]:
import keyring, os

os.environ["KAGGLE_USERNAME"] = keyring.get_password("KAGGLE_USERNAME", "kaggle_username")
os.environ["KAGGLE_KEY"] = keyring.get_password("KAGGLE_KEY", "kaggle_key")

In [17]:
import requests
from pathlib import Path
from tqdm import tqdm
import zipfile
import os
import json

# Where to save data
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

KAGGLE_USERNAME = os.environ["KAGGLE_USERNAME"]
KAGGLE_KEY = os.environ["KAGGLE_KEY"]

def download_file(file_name):
    url = f"https://www.kaggle.com/api/v1/competitions/data/download/amex-default-prediction/{file_name}"
    auth = (KAGGLE_USERNAME, KAGGLE_KEY)
    headers = {"User-Agent": "Mozilla/5.0"}
    zip_path = DATA_DIR / f"{file_name}.zip"

    print(f"⬇️  Downloading {file_name}...")

    with requests.get(url, stream=True, auth=auth, headers=headers) as r:
        r.raise_for_status()
        total_size = int(r.headers.get("content-length", 0))
        with open(zip_path, "wb") as f:
            for chunk in tqdm(r.iter_content(chunk_size=1024 * 1024),  # 1MB chunks
                              total=total_size // (1024 * 1024),
                              unit="MB"):
                f.write(chunk)

    print(f"✅ Downloaded {file_name}. Extracting...")

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(DATA_DIR)

    os.remove(zip_path)
    print(f"🧹 Cleaned up {file_name}.zip")

# Files to grab
files = ["train_data.csv", "train_labels.csv", "sample_submission.csv"]
for f in files:
    download_file(f)

⬇️  Downloading train_data.csv...


6827MB [14:26,  7.88MB/s]                            


✅ Downloaded train_data.csv. Extracting...
🧹 Cleaned up train_data.csv.zip
⬇️  Downloading train_labels.csv...


17MB [00:03,  4.86MB/s]                        


✅ Downloaded train_labels.csv. Extracting...
🧹 Cleaned up train_labels.csv.zip
⬇️  Downloading sample_submission.csv...


33MB [00:08,  4.08MB/s]                        


✅ Downloaded sample_submission.csv. Extracting...
🧹 Cleaned up sample_submission.csv.zip
